# Week 4 – Virtual Tour / Three.js Export
Run the incremental SfM, export a lightweight bundle for the web, and generate a minimal Three.js viewer.

In [13]:
from pathlib import Path
import sys

# Locate project root and assets
PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "assets").exists() and (PROJECT_ROOT.parent / "assets").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

ASSETS_DIR = PROJECT_ROOT / "assets"
OUTPUT_DIR = PROJECT_ROOT / "outputs" / "reconstruction"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

ASSETS_DIR, OUTPUT_DIR

(PosixPath('/Users/muhammadabdullahirfan/Desktop/Uni/Sem_5/CV/submission/Computer-Vision-3D-Scene-Reconstruction/assets'),
 PosixPath('/Users/muhammadabdullahirfan/Desktop/Uni/Sem_5/CV/submission/Computer-Vision-3D-Scene-Reconstruction/outputs/reconstruction'))

In [14]:
from src.multi_view_sfm import run_incremental_sfm

result = run_incremental_sfm(
    asset_dir=ASSETS_DIR,
    detector="SIFT",
    ratio_thresh=0.75,
    refine=True,
    min_correspondences=12,
    return_tracks=True,
)

result.stats

{'images': 14,
 'registered': 14,
 'skipped': [],
 'points': 2768,
 'pose_inliers': {0: 0,
  1: 0,
  2: 147,
  3: 255,
  4: 221,
  5: 207,
  6: 163,
  7: 171,
  8: 134,
  9: 34,
  10: 57,
  11: 31,
  12: 69,
  13: 83},
 'retriangulated': 1271}

In [15]:
import json
import numpy as np
from src.interpolation import rotation_to_quaternion


def camera_center(R: np.ndarray, t: np.ndarray) -> list[float]:
    return (-R.T @ t).ravel().tolist()


bundle = {
    "intrinsics": result.K.tolist(),
    "images": [p.name for p in result.image_paths or []],
    "cameras": [],
    "points": [],
}

for idx, cam in enumerate(result.poses):
    if not cam.registered:
        continue
    bundle["cameras"].append(
        {
            "index": idx,
            "image": cam.image_path.name if cam.image_path else None,
            "R": cam.R.tolist(),
            "t": cam.t.ravel().tolist(),
            "center": camera_center(cam.R, cam.t),
            "quat": rotation_to_quaternion(cam.R).tolist(),
        }
    )

for pt, rgb in zip(result.points_3d, result.colors_rgb.astype(int)):
    bundle["points"].append({"xyz": [float(v) for v in pt.tolist()], "rgb": [int(c) for c in rgb.tolist()]})

json_path = OUTPUT_DIR / "virtual_tour_data.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(bundle, f, indent=2)
print(f"Wrote {len(bundle['cameras'])} cameras and {len(bundle['points'])} points -> {json_path}")

html_path = OUTPUT_DIR / "virtual_tour_viewer.html"
html = """<!doctype html>
<html lang='en'>
<head>
  <meta charset='UTF-8'>
  <meta name='viewport' content='width=device-width, initial-scale=1.0'>
  <title>Virtual Tour Viewer</title>
  <style>
    body, html { margin: 0; padding: 0; overflow: hidden; background: #0a0a0f; }
    #info { position: absolute; top: 12px; left: 12px; color: #f0f0f0; font-family: monospace; z-index: 1; }
  </style>
  <link rel="icon" href="data:,">
  <script type="importmap">{
    "imports": {
      "three": "https://cdn.jsdelivr.net/npm/three@0.160.0/build/three.module.js",
      "three/examples/jsm/": "https://cdn.jsdelivr.net/npm/three@0.160.0/examples/jsm/"
    }
  }</script>
</head>
<body>
  <div id='info'>Drag: orbit, Scroll: zoom</div>
  <canvas id='c'></canvas>
  <script type='module'>
    import * as THREE from 'three';
    import { OrbitControls } from 'three/examples/jsm/controls/OrbitControls.js';

    const canvas = document.getElementById('c');
    const renderer = new THREE.WebGLRenderer({canvas, antialias: true});
    const scene = new THREE.Scene();
    scene.background = new THREE.Color(0x0a0a0f);
    const camera = new THREE.PerspectiveCamera(60, 2, 0.1, 5000);
    const controls = new OrbitControls(camera, renderer.domElement);

    fetch('virtual_tour_data.json').then(r => r.json()).then(data => {
      const pts = data.points;
      const positions = new Float32Array(pts.length * 3);
      const colors = new Float32Array(pts.length * 3);
      let i = 0;
      for (const p of pts) {
        positions[3*i] = p.xyz[0];
        positions[3*i+1] = p.xyz[1];
        positions[3*i+2] = p.xyz[2];
        colors[3*i] = p.rgb[0] / 255;
        colors[3*i+1] = p.rgb[1] / 255;
        colors[3*i+2] = p.rgb[2] / 255;
        i++;
      }
      const geom = new THREE.BufferGeometry();
      geom.setAttribute('position', new THREE.BufferAttribute(positions, 3));
      geom.setAttribute('color', new THREE.BufferAttribute(colors, 3));
      const mat = new THREE.PointsMaterial({size: 2.2, vertexColors: true, sizeAttenuation: true});
      scene.add(new THREE.Points(geom, mat));

      const camGeom = new THREE.SphereGeometry(4, 10, 10);
      const camMat = new THREE.MeshBasicMaterial({color: 0xffffff});
      for (const c of data.cameras) {
        const m = new THREE.Mesh(camGeom, camMat);
        m.position.set(c.center[0], c.center[1], c.center[2]);
        scene.add(m);
      }

      geom.computeBoundingBox();
      const bb = geom.boundingBox;
      const center = new THREE.Vector3();
      bb.getCenter(center);
      const size = new THREE.Vector3();
      bb.getSize(size);
      const radius = Math.max(size.x, size.y, size.z) * 0.7;
      camera.position.set(center.x, center.y, center.z + radius * 1.5);
      controls.target.copy(center);
      controls.update();
    });

    function resizeRenderer() {
      const w = window.innerWidth;
      const h = window.innerHeight;
      renderer.setSize(w, h, false);
      camera.aspect = w / h;
      camera.updateProjectionMatrix();
    }
    window.addEventListener('resize', resizeRenderer);
    resizeRenderer();

    function render() {
      renderer.render(scene, camera);
      requestAnimationFrame(render);
    }
    render();
  </script>
</body>
</html>"""

with open(html_path, "w", encoding="utf-8") as f:
    f.write(html)
print(f"Wrote {html_path}")


Wrote 14 cameras and 2768 points -> /Users/muhammadabdullahirfan/Desktop/Uni/Sem_5/CV/submission/Computer-Vision-3D-Scene-Reconstruction/outputs/reconstruction/virtual_tour_data.json
Wrote /Users/muhammadabdullahirfan/Desktop/Uni/Sem_5/CV/submission/Computer-Vision-3D-Scene-Reconstruction/outputs/reconstruction/virtual_tour_viewer.html


## How to view the Three.js app
1. Run the notebook cells above.
2. Start a local server from the reconstruction output folder:
   ```bash
   cd outputs/reconstruction
   python -m http.server 8000
   ```
3. Open `http://localhost:8000/virtual_tour_viewer.html` in your browser.

Controls: drag to orbit, scroll to zoom; the point cloud is colored by the source images and camera centers are shown as white spheres.